# Chapter 8: Python Optimization
___

## Course plan

**Introduction**
    1. Basics: what is an optimization problem, how to solve, and line search
**Unconstrained optimization**
    2. Direct search methods: Hooke&Jeeves and Powell's methods 
    3. Steepest Descent and Newton's method for unconstrained optimization
    4. Example of using available software: `scipy.optimize`
**Constrained optimization**
    5. Indirect methods for constrained optimization
    6. Direct methods for constrained optimization
    7. Optimality conditions
    8. Methods using optimality conditions for constrained optimization
    9. Algebraic modeling languages, especially Pyomo
**Multiobjective optimization**
    10. What is multiobjective optimization
    11. How to solve multiobjective optimization problems
**Applications of optimization**
    12. How to find and read scientific papers in the field
    13. Students finding applications of optimization themselves
**Wrapping up**
    14. Further topics and current research in optimization
    15. Review

## 1. Optimization Basics

### 1.1 What is an optimization problem?

A general mathematical formulation for **the optimization problems** is
$$
\begin{align} \
\min \quad &f(x)\\
\textit{s.t.} \quad & g_j(x) \le 0\text{ for all }j=1,\ldots,J\\
& h_k(x) = 0\text{ for all }k=1,\ldots,K\\
&x\in \mathbb R^n.
\end{align}
$$

The above problem can be expressed as 
>Find an $x\in \mathbb R^n$ such that $g_j(x)\le 0$ for all $j=1,\ldots,J$ and $h_k(x)=0$ for all $k=1,\ldots,K$, and there does not exist $x'\in \mathbb R^n$ satisfying $f(x')<f(x)$ and $g_j(x')\geq 0$ for all $j=1,\ldots,J$, $h_k(x')=0$ for all $k=1,\ldots,K$.

### 1.2 Some important concepts

There are three main components to an optimization problem:
* the variables $x$ are called the **decision variables (决策变量)**,
* the equalities and inequalities $g_j(x)\geq 0$ and $h_k(x)=0$ are called the **constraints (约束)**,
* the funtion $f(x)$ is called the **objective function (目标函数)**.

Values of decision variables $x^*$ are called **solutions(解)** and a solution is called
* **feasible (可行解)** if $g_j(x^*)\geq 0$ for all $j=1,\ldots,J$, $h_k(x^*)=0$ for all $k=1,\ldots,K$,
* **locally optimal (局部最优解)** if $x^*$ is feasible and there exists $r>0$ such that there does not exist a feasible solution $x'\in \operatorname{B}(x^*,r)$ such that $f(x')<f(x^*)$, and
* **optimal (最优解)** if $x^*$ is feasible and there does not exist a feasible solution $x'$ such that $f(x')<f(x^*)$.

The problem is called
* **linear/nonlinear (线性/非线性)** if the objective function and the constraints of the problem are/are not affinely linear,
* **multi/unimodal (多模/单模)** if the problem has/does not have more than one local optimum,
* **convex/nonconvex (凸/非凸)** if the objective and the constraints are that,
* **continuous/differentiable/twice-differentiable, etc (连续/可微/二次可微)** if the objective and the constraints are that.

## 2. How to solve optimization problems?

### 2.1 Iterative vs. non-iterative methods

Optimal solutions to some optimization problems can be found by defining an explicit formula for it. For example, if *the objective function is twice continuously differentiable （二阶连续可导）and there are no constraints （无约束）*, the optimal solution (if exists) can be found by calculating all the zero-points of the gradient and finding the best one of those. In this kind of cases, the optimization problem can be solved using **non-iterative methods.**

*When the problem has constraints, or the problem is in some other way not-well behaved*, ** iterative methods** are needed. In iterative methods, the solving the optimizaiton problem starts from a **initial** solution and then tries to *improve the solution iteratively*. The optimization algorithm chooses how the solution is changed at each iteration.

Often the methods cannot guarantee a (global) optimum, but instead **we need to satisfy ourselves with a local optimum**. In addition, it is usually not possible to find the actual optimal solution, but instead **an approximation of the optimal solution**. A feasible solution $x^*$ is called an approximation of a local optimum $x^{**}$ with quality $r>0$, when $\|x^*-x^{**}\|\leq r$.

### 2.2 Line search (线性搜索)

For a box-constrained optimization problem
$$
\min_x\;f(x)\quad \textit{s.t.}\;x \in [a,b]
$$
where $a,b\in\mathbb R$.

Let's illustrate how to find an approximation of a local optimum using an example below.

In [ ]:
def f(x):
    """ An example optimization problem. """
    return 2+(1-x)**2

print "The value of the objective function at 0 is", f(0)

### $\S$ Example: Line search with fixed steps
**Input**: $r>0$.  
**Output**: $x^*$.<br>
**LineSearch**: 
1. Initiate $x \in [a, b]$
2. Loop:
```
    If f(x+r) > f(x):
        stop, and the locally optimal solution is x 
    x = x + r
```

In [ ]:
def fixed_linesearch(a,b,f,r):
    x = a
    while f(x)>f(x+r) and x+r<b:
        x=x+r
    return x

In [ ]:
x = fixed_linesearch(0.0, 3.0, f, 1e-3)
print "Line search finds that the optimal point in [0.0, 3.0] is", x, ", and the optimum is", f(x)

In [ ]:
%timeit fixed_linesearch(0.0, 3.0, f, 1e-3)

### 2.3 Bisection search (二分线性搜索)

**Input:** $r>0$ and $\epsilon > 0$.  
**Output:** $x^*$.<br/>
**Bisection Search**:
```
Set x=a and y=b
while y-x > 2*r:
    if f((x+y)/2+eps) > f((x+y)/2-eps):
        set y = (x+y)/2
    otherwise:
        set x = (x+y)/2
```

In [ ]:
def bisection(a,b,f,L,epsilon):
    x = a
    y = b
    while y-x>2*L:
        if f((x+y)/2+epsilon)>f((x+y)/2-epsilon):
            y=(x+y)/2+epsilon
        else:
            x = (x+y)/2-epsilon
    return (x+y)/2

In [ ]:
x = bisection(0.0, 3.0, f, 1e-3, 1e-4)
print "Bisection line search finds that the optimal point in [0.0, 3.0] is", x, ", and the optimum is", f(x)

In [ ]:
%timeit bisection(0.0, 3.0, f, 1e-3, 1e-4)

### 2.4 Golden-section line search

#### Golden section

Let $a<c<b$ be such that $\frac{b-a}{c-a}=\frac{c-a}{b-c}$. Then it is said that the point $c$ devides interval $[a,b]$ in the ratio of golden section (from the left, mirror from the right). Note that $c=a+\frac{\sqrt{5}-1}2(b-a)\approx a+0.618(b-a)$.

There is a theorem that if $a<c<d<b$ and both points divide the interval $[a,b]$ in the ratio of golden section (from right and left), then the point $c$ divides the interval $[a,d]$ in the ratio of golder ration from the left.

**Input:** the quality $r>0$ of the approximation of the local optimum.  
**Output:** an approximation of the local optimum with quality $r$.<br/>
**GoldenSection:**
```
Set x=a and y=b
while y-x>2*r:
    Get left and right golden division point c < d for the interval [x,y]
    If f(d) > f(c): 
        set y=d
    otherwise:
        set x=c
return (x+y)/2
```

In [ ]:
import math
def golden_section(a, b, f, r):
    x, y = a, b
    while y - x > 2*r:
        c = y - (math.sqrt(5)-1)*(y-x)/2
        d = x + (math.sqrt(5)-1)*(y-x)/2
        if f(d) > f(c):
            y = d
        else:
            x = c
    return (x+y)/2

In [ ]:
x = golden_section(0.0, 3.0, f, 1e-3)
print "Golden section line search finds that the optimal point in [0.0, 3.0] is", x, ", and the optimum is", f(x)

In [ ]:
%timeit golden_section(0.0, 3.0, f, 1e-3)

## 3. Direct search: Hooke&Jeeves and Powell's method

We will start studying functions of multiple variables by studying unconstrained optimization problems
$$
\begin{align}
\min \quad &f(x)\\
\text{s.t.}\quad &x\in \mathbb R^n
\end{align}  
$$

And here is an example:
$$
\begin{align}
\min \quad & (x_1-10)^2+(x_2+5)^2+x_1^2\\
\text{s.t.}\quad &x_1,x_2\in\mathbb R
\end{align}  
$$
This problem is unconstrained, because there are no constraints.

Now we need to redefine a function in Python, which is a two-variable function 
$$f:(x_1,x_2)\to (x_1-10)^2+(x_2+5)^2+x_1^2$$

In [ ]:
def f_simple(x):
    return (x[0] - 10.0)**2 + (x[1] + 5.0)**2+x[0]**2
print "At point (3,-8) the value of the function is",f_simple([3,-8])

And we can also plot the function:

In [ ]:
import numpy as np
from pylab import meshgrid
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

def plot_2d_function(lb1,lb2,ub1,ub2,f):
    x = np.arange(lb1,ub1,0.1)
    y = np.arange(lb2,ub2,0.1)
    X,Y = meshgrid(x, y) # grid of point
    Z = [f([x,y]) for (x,y) in zip (X,Y)] # evaluation of the function on the grid
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z)
    return plt

In [ ]:
plot_2d_function(3,-7,7,-3,f_simple)

### 3.1 Direct search methods

**Direct search methods** (also called pattern search methods) rely only on the function values to find a (local) optimum. Direct search methods consist of a set of  
1. **Exploratory moves** that acquire information about the function $f$ in the neighbourhood of current solution, and
2. **Pattern moves** that attempt to speed up the search using the information acquired in the exploratory moves.

### 3.2 Hooke&Jeeves algorithm

**input:** a minimum step length $L>0$, initial step length $\epsilon_0$, constant $0<\delta<1$ for reducing the step length, exploratory step multiplier $\gamma>1$, and starting solution $x_0$<br/>
**output:** an approximation of a local optimum (no guarantees of quality in general cases)  
```
set eps=eps0
set x=x0
while eps > L:
    for each coordinate direction i:
        find the smallest of function values by incrementing and reducing the variable value in that coordinate by eps, let this value be xi*
    if x==(x1*,...,xn*):
        reduce eps = delta*eps
    else:
        if f(x1*,...,xn*) < f(x+gamma*((x1*,...,xn*)-x)):
            set x = (x1*,...xn*)
        else:
            set x = x+gamma*((x1*,...,xn*)-x)
return x
        
```
Thus, 
* the **exploratory step** of Hooke&Jeeves is performed by incrementing and reducing the variable to each coordinate direction and 
* the **pattern move** is just a multiplication of the exploratory move.

In [ ]:
import copy #Copying vectors
import numpy as np #Import vector calculus and much more!
def hookejeeves(L,epsilon0,delta,gamma,x0,f):
    #Set up the initial values
    epsilon = epsilon0
    x = np.array(x0)
    #Loop while step length greater than L:
    while epsilon>L:
        #our exploratory move is initially [0,..,0]
        xtest = np.zeros(len(x))
        for coordinate in range(len(x)):
            #First points to be explored are the all x, to be changed
            exp_points = [copy.copy(x) for _ in range(3)] #points to be explored
            #Change exp_points[0] and exp_points[1] to reflect
            #moving along the coordinate
            exp_points[0][coordinate]-=epsilon
            exp_points[1][coordinate]+=epsilon
            #Assign the function values given by exp_points to a list
            f_exp_points = [f(exp_point) for exp_point in exp_points]
            #pick the smallest one of them
            min_value = min(f_exp_points)
            #The exploratory move to the coordinate direction is given by the
            #move giving the smallest value of f
            xtest[coordinate] = exp_points[f_exp_points.index(min_value)][coordinate] #The coordinate value is the one where the minimum is attained
        #If no move at all, then reduce the exploratory move step size
        if all(xtest==x):
            epsilon = delta*epsilon
        else:
            #if exploratory move is better than pattern move
            if f(xtest)<f(x+gamma*(xtest-x)):
                #...set x as the exploratory move
                x = xtest
            else:
                #Otherwise we take the pattern move
                x = x+gamma*(xtest-x)
    return x

In [ ]:
L = 0.001
epsilon0 = 1
delta = 0.1
gamma = 2.0
start = [0.0,0.0]

In [ ]:
x = hookejeeves(L,epsilon0,delta,gamma,start,f_simple)
print "Optimal solution is", x, "and the optimal objective value is", f_simple(x) 

In [ ]:
%timeit hookejeeves(L,epsilon0,delta,gamma,start,f_simple)

Let's have a look at the search process:

In [ ]:
%pylab inline
import matplotlib.pyplot as plt

def plot_2d_steps(steps,start):
    myvec = np.array([start]+steps).transpose()
    plt.plot(myvec[0,],myvec[1,],'ro')
    for label,x,y in zip([str(i) for i in range(len(steps)+1)],myvec[0,],myvec[1,]):
        plt.annotate(label,xy = (x+.2, y))
    plt.xlim(min(myvec[0])-1, max(myvec[0])+1)
    plt.ylim(min(myvec[1])-1, max(myvec[1])+1)
    return plt

In [ ]:
import copy #Copying vectors
import numpy as np #Import vector calculus and much more!
def hookejeeves_savesteps(L,epsilon0,delta,gamma,x0,f):
    epsilon = epsilon0
    x = np.array(x0)
    steps = []
    while epsilon>L:
        xtest = np.zeros(len(x))
        for coordinate in range(len(x)):
            exp_points = [copy.copy(x) for _ in range(3)] #points to be explored
            exp_points[0][coordinate]-=epsilon
            exp_points[1][coordinate]+=epsilon
            f_exp_points = [f(exp_point) for exp_point in exp_points]
            min_value = min(f_exp_points)
            xtest[coordinate] = exp_points[f_exp_points.index(min_value)][coordinate] 
            #The coordinate value is the one where the minimum is attained
        if all(xtest==x):
            epsilon = delta*epsilon
        else:
            if f(xtest)<f(x+gamma*(xtest-x)):
                x = xtest
            else:
                x = x+gamma*(xtest-x)
            steps.append(x)
    return x,steps

In [ ]:
L = 0.001
epsilon0 = 1.0
delta = 0.1
gamma = 0.1
start = [0.,0.]
(x,steps) = hookejeeves_savesteps(L,epsilon0,delta,gamma,start,f_simple)

In [ ]:
min([start[0], map(lambda x: x[0], steps)])

In [ ]:
max([start[0], map(lambda x: x[0], steps)])

In [ ]:
plot_2d_steps(steps,start)

### 3.3 Powell's method

Powell's method is similar to Hooke&Jeeves, but the first step in exploratory moves is taken to the direction of the last pattern move. This speeds up the convergence in most cases.

In [ ]:
import copy
import numpy as np
def powell_savesteps(L,epsilon0,delta,gamma,x0,f):
    epsilon = epsilon0
    exp_direction = np.array([0,1])
    x = np.array(x0)
    steps = []
    while epsilon>L:
        exp_direction=epsilon*exp_direction
        #Comparing among exploratory points to firtst exploratory direction:
        if f(x+exp_direction)<f(x):
            exp_step1=exp_direction
        elif f(x-exp_direction)<f(x):
            exp_step1=-exp_direction
        else:
            exp_step1 = np.zeros(2)
        #The following only works in 2d!!
        exp_direction2 = np.array([exp_direction[1],-exp_direction[0]])
        if f(x+exp_direction2)<f(x):
            exp_step2=exp_direction2
        elif f(x-exp_direction2)<f(x):
            exp_step2=-exp_direction2
        else:
            exp_step2 = np.zeros(2)
        if all(exp_step1+exp_step2==0):
            epsilon = delta*epsilon
        else:
            if f(x+(exp_step1+exp_step2))<f(x+gamma*(exp_step1+exp_step2)):
                x = x+(exp_step1+exp_step2)
            else:
                x = x+gamma*(exp_step1+exp_step2)
            steps.append(x)
            exp_direction = (exp_step1+exp_step2)/np.linalg.norm(exp_step1+exp_step2)
    return x,steps

In [ ]:
L = 0.001
epsilon0 = 10
delta = 0.01
gamma = 2.0
start = [-2.,1.]
(x,steps) = powell_savesteps(L,epsilon0,delta,gamma,start,f_simple)
print "Optimal solution is" + str(x)

Let's see how it works:

In [ ]:
plot_2d_steps(steps,start)

## 4. Steepest descent and Newton's method

Before that, we need to import a python automated differentiation package `ad`:

In [ ]:
import ad

In [ ]:
grad_f, hess_f = ad.gh(f_simple)
print "At the point (1,2) gradient is", grad_f([1,2]), "and hessian is", hess_f([1,2])

Let's write a function to view the gradients:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from pylab import meshgrid
def visualize_gradient(f,point,x_lim,y_lim):
    grad_point = np.array(ad.gh(f)[0](point))
    grad_point = grad_point/np.linalg.norm(grad_point)
    X,Y,Z = point[0],point[1],f(point)
    U,V,W = grad_point[0],grad_point[1],0
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(x_lim[0],x_lim[1],0.1)
    y = np.arange(y_lim[0],y_lim[1],0.1)
    X2,Y2 = meshgrid(x, y) # grid of point
    Z2 = [f([x,y]) for (x,y) in zip (X2,Y2)] # evaluation of the function on the grid
    surf = ax.plot_surface(X2, Y2, Z2,alpha=0.5)
    ax.quiver(X,Y,Z,U,V,W,color='red',linewidth=1.5)
    return plt

In [ ]:
visualize_gradient(f_simple,[1,-2],[0,10],[-10,0])

With the `lambda` function we can easily visualize gradients of various functions:

In [ ]:
visualize_gradient(lambda x:3*x[0]+x[1],[1,0],[0,2],[0,2])

### 4.1 Prototype algorithm for the steepest descent and Newton's methods

**Input:** function $f$ to be optimized, starting point $x_0$, step length rule $alpha$, stopping rule $stop$<br/>  
**Output:** A solution $x^*$ that is close to a locally optimal solution
```
Initiate x = x0
set f_old as a big number and f_new as f(x0)
while a stopping criterion has not been met:
    f_old = f_new
    determine search direction d_h according to the method
    determine the step length alpha
    set x = x + alpha *d_h
    f_new = f(x)
return x
```

The way to determine search direction distinguishes steepest descent algorithm and the Newton algorithm. Different stopping rules and step sizes can be mixed and matched with both algorithms.

### 4.2 Steepest descent: for unconstrained problem

In the steepest descent algorithm, the search direction is determined by the __negative of the gradient $-\nabla f(x)$__.

In [ ]:
import numpy as np
import ad
def steepest_descent(f,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        d = -np.array(ad.gh(f)[0](x))
        x = x+d*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [ ]:
start = [2.0,-10.0]
(x_value,f_value,steps) = steepest_descent(f_simple,start,0.2,0.0001)
print "Optimal solution is ", x_value

We can plot the solution path here:

In [ ]:
plot_2d_steps(steps,start)

### 4.3 Newton's method: Unconstrained problems with existing gradient and hessian

In Newton's method, the search direction is determined by both gradient and Hessian of the objective function, as $-H^{-1}\nabla f(x)$.

In [ ]:
def newton(f,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        H_inv = np.linalg.inv(np.matrix(ad.gh(f)[1](x)))
        d = (-H_inv*(np.matrix(ad.gh(f)[0](x)).transpose())).transpose()
        #Change the type from np.matrix to np.array so that we can use it in our function
        x = np.array(x+d*step)[0]
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [ ]:
start = [2.0,-10.0]
(x_value,f_value,steps) = newton(f_simple,start,0.5,0.01)
print "Optimal solution is ",x_value

In [ ]:
plot_2d_steps(steps,start)

## 5. Applying `scipy.optimize`

In [ ]:
from scipy.optimize import minimize

### 5.1 Nelder-Mead: Simplex method

The Nelder-Mead method does not rely on the gradient and hessian of the objective function.

In [ ]:
res = minimize(f_simple,[0,0],method='Nelder-Mead', 
         options={'disp': True})
print res.x

From the result, we can see that:
* The optimization process has reached the optimum, and terminated successfully.
* The minimum objective value is 50.00
* It took 99 iterations to reach the minimum.
* The function `f_simple` was evaluated 189 times.

### 5.2 Conjugate gradient

The nonlinear conjugate gradient algorithm was proposed by Polak and Ribiere, which is a variant of the Fletcher-Reeves method. The gradient (`jac`) can either be estimated numerically, or provided by an analytical function.

#### 5.2.1 Estimating the gradient numerically

In [ ]:
import numpy as np
res = minimize(f_simple, [0,0], method='CG', #Conjugate gradient method
               options={'disp': True})
print res.x

#### 5.2.2 Specifying the gradient function

In [ ]:
import ad
res = minimize(f_simple, [0,0], method='CG', #Conjugate gradient method
               options={'disp': True}, jac=ad.gh(f_simple)[0])
print res.x

### 5.3 Newton's Conjugate gradient method

Newton's conjugate gradient method, a.k.a truncated Newton's method. It uses a conjugate gradient method to the compute the search direction. See also TNC method for a box-constrained minimization with a similar algorithm.

e Newton-CG algorithm requires both the gradient (`jac`) and the Hessian (`hess, hessp`). The __Jacobian__ should be provided as function calls, while the __Hessian__ can be estimated numerically. 

In [ ]:
res = minimize(f_simple, [0,0], method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_simple)[0])
print res.x

We can also provide the __Hessian__:

In [ ]:
res = minimize(f_simple, [0,0], method='Newton-CG', #Newton-CG method
               options={'disp': True},jac=ad.gh(f_simple)[0],
               hess=ad.gh(f_simple)[1])
print res.x

### 5.4 Optimization for function with a single variable 

#### 5.4.1 Line search

The line search algorithm is as we mentioned previously. 

In [ ]:
def f_singlevar(x):
    return 2+(1-x)**2

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
minimize_scalar?

In [ ]:
minimize_scalar-brent

In [ ]:
minimize_scalar(f_singlevar, bounds=[0.0,3.0], method='Bounded', options={'disp': True})

#### 5.4.2 Brent method

Brent's algorithm is used to find a local minimum. The algorithm uses inverse parabolic interpolation when possible to speed up convergence of the golden section method.

In [ ]:
minimize_scalar(f_singlevar, bounds=[0,3], method='Brent')

#### 5.4.3 Golden section method

In [ ]:
minimize_scalar(f_singlevar, bounds=[0,3], method='Golden')

## 6. Constrained optimization

Now we will turn to study the constrained optimizaton problems i.e., the full problem
$$
\begin{align} \
\min \quad &f(x)\\
\text{s.t.} \quad & g_j(x) \geq 0\text{ for all }j=1,\ldots,J\\
& h_k(x) = 0\text{ for all }k=1,\ldots,K\\
&a_i\leq x_i\leq b_i\text{ for all } i=1,\ldots,n\\
&x\in \mathbb R^n,
\end{align}
$$
where for all $i=1,\ldots,n$ it holds that $a_i,b_i\in \mathbb R$ or they may also be $-\infty$ of $\infty$.

For example, we can have an optimization problem
$$
\begin{align} \
\min \quad &x_1^2+x_2^2\\
\text{s.t.} \quad & x_1+x_2-1\geq 0\\
&-1\leq x_1\leq 1, x_2\leq 3.\\
\end{align}
$$

The optimization problem can be defined as a Python function:

In [ ]:
import numpy as np
def f_constrained(x):
    return np.linalg.norm(x)**2,[x[0]+x[1]-1],[]

Now we can call the function:

In [ ]:
(f_val,ieq,eq) = f_constrained([1,0])
print "Value of f is", f_val
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "

We can also check the feasibility of the solution:

In [ ]:
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution [1, 0] is feasible"
else:
    print "Solution [1,0] is infeasible"

### Indirect and direct methods for constrained problems

There are two categories of methods for constrained optimization: Indirect and direct methods. The main difference is that
1. __Indirect methods__ convert the constrained optimization problem into a single or a sequence of unconstrained optimization problems, that are then solved. Often, the intermediate solutions do not need to be feasible, the sequence of solutions converges to a solution that is optimal (and, thus, feasible).
2. __Direct methods__ deal with the constrained optimization problem directly. In this case, the intermediate solutions are feasible.

### 6.1 Indirect approach: Penalty function method

The IDAE for __penalty function method__ is to penalize the violations of the constraints by embedding the inequality and equality constraints as the regularization terms into a unconstrained optimization problem.

Let, $\alpha(x):\mathbb R^n\to\mathbb R$ be a function so that 
* $\alpha(x)=0$ for all feasible $x$
* $\alpha(x)>0$ for all infeasible $x$.

Define optimization problems
$$
\begin{align} \
\min \qquad &f(x)+\lambda\alpha(x)\\
\text{s.t.} \qquad &x\in \mathbb R^n
\end{align}
$$
for $\lambda>0$ and $x_p$ be the optimal solutions of these problems.

In this case, the optimal solutions $x_p$ converge to the optimal solution of the constrained problem, when $\lambda \to\infty$, if such solution exists.

The known candidates for penalty functions include
* $h_k(x)^2$ for equality constraints,
* $\left(\min\{0,g_j(x)\}\right)^2$ for inequality constraints.

Using the above penalties, we can build our regularization term:

In [ ]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])+sum([eq_k**2 for eq_k in eq])

and also the penalized function (the final unconstrained objective function):

In [ ]:
def penalized_function(x,f,lam):
    return f(x)[0] + lam*alpha(x,f)

Here is an example:

In [ ]:
penalized_function([-1,0],f_constrained,100)

And this unconstrained optimization problem can now be solved using __Nelder-Mead__ approach:

In [ ]:
res = minimize(lambda x:penalized_function(x,f_constrained,10000),
         [0,0],method='Nelder-Mead', 
         options={'disp': True})
print res

In [ ]:
(f_val,ieq,eq) = f_constrained(res.x)
print "Value of f is "+str(f_val)
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "

if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution is feasible"
else:
    print "Solution is infeasible"

### But how to set the penalty term $\lambda$?

The penalty term should
* be large enough in order for the solutions be close enough to the feasible region, but
* not be too large to
  * cause numerical problems, or
  * cause premature convergence to non-optimal solutions because of relative tolerances.

Usually, the penalty term is either
* set as big as possible without causing problems (hard to know), or
* updated iteratively.

### 6.2 Barrier function method: Indirect method

The IDEA for barrier function method is to prevent leaving the feasible domain by setting the objective value to be $\infty$ once outside the feasible region.

This method is __only applicable to__ problems with inequality constraints and for which the set 
$$\{x\in \mathbb R^n: g_j(x)>0\quad \forall j=1,\ldots,J\}$$
is non-empty.

Let $\beta:\{x \in \mathbb R^n: g_j(x)>0\quad \forall j=1,\ldots,J\}\to \mathbb R$ be a function so that $\beta(x)\to \infty$, when $x\to\partial\{x\in \mathbb R^n: g_j(x)>0\quad \forall j=1,\ldots,J\}$, where $\partial A$ is the boundary of the set $A$. Now, define optimization problem 
$$
\begin{align}
\min \qquad & f(x) + \lambda \beta(x)\\
\text{s.t. } \qquad & x\in \{x\in \mathbb R^n: g_j(x)>0\quad \forall j=1,\ldots,J\}.
\end{align}
$$
and let $x_p$ be the optimal solution of this problem (which we assume to exist for all $\lambda>0$).

In this case, $x_p$ converges to the optimal solution of the problem (if it exists), when $\lambda \to 0^+$ (i.e., $\lambda$ converges to zero from the right).

One of the good choices barrier algorithm is $\frac{1}{g_j(x)}$, where $g_j(x)$ is the $j$-th inequation constraint.

In [ ]:
def beta(x,f):
    _,ieq,_ = f(x)
    try:
        value = sum([1/max(0, ieq_j) for ieq_j in ieq]) 
    except ZeroDivisionError:
        value = float("Inf")
    return value

In [ ]:
def function_with_barrier(x,f,lam):
    return f(x)[0]+lam*beta(x,f)

In [ ]:
res = minimize(lambda x: function_with_barrier(x,f_constrained,0.00000000000001),
         [1,1], method='Nelder-Mead', options={'disp':True})
print res.x

In [ ]:
(f_val,ieq,eq) = f_constrained(res.x)
print "Value of f is "+str(f_val)
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution is feasible"
else:
    print "Solution is infeasible"

### Notes about using penalty and barrier function methods

* It is worthwile to consider whether feasibility can be compromized. If the constraints do not have any tolerance, then barrier function method should be considered.
* Also barrier methods parameter can be set iteratively
* Penalty and barrier functions should be chosen so that they are differentiable (thus $x^2$ above)
* In both methods, the minimum is attained at the limit.
* Different penalty and barrier parameters can be used for differnt constraints, even for same problem.

## 6.4 Direct methods

### Feasible descent directions

Let $S\subset \mathbb R^n$ ($S\neq \emptyset$ closed 非空闭集) and $x^*\in S$. 

**Definition:** The set
$$ D = \{d\in \mathbb R^n: d\neq0,x^*+\alpha d\in S \text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$
is called the __cone of feasible directions of $S$ in $x^*$ （在$x^*$点处的可行方向锥）__.

**Definition:** The set 
$$ F = \{d\in \mathbb R^n: f(x^*+\alpha d)<f(x^*)\text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$
is called the __cone of descent directions (在$x^*$处的下降方向锥)__.

**Definition:** The set $F\cap D$ is called the __cone of feasible descent directions (可行下降方向锥)__.

### $\S$ Theorem
Consider an optimization problem 
$$
\begin{align}
\min &\  f(x)\\
\text{s.t. }&\ x\in S
\end{align}
$$
and let $x^*\in S$. Now if $x^*$ is a local minimizer **then** the set of feasible descent directions $F\cap D$ is empty.

### Idea for the methods of feasible descent directions

1. Assume a feasible solution $x$.
2. Find a feasible descent direction $d\in D\cap F$.
3. Determine the step length to the direction $d$
4. Update $x$ accordingly.

### 6.5 Rosen's projected gradient method

Here is a problem with __linear equality constraints__:
$$
\begin{align}
\min\quad &f(x)\\
\text{s.t.}\quad &Ax=b
\end{align}
$$
where $A \in \mathbb R^{m\times n}$ ($m\leq n$) and $b$ is a vector.

Let $x$ be a feasible solution to the above problem.

It holds that:

> $d$ is a feasible direction *if and only if* $Ad=0$.

Thus, the gradient $\nabla f(x)$ is a feasible descent direction, if 
$$ A\nabla f(x)=0.$$

This may or may not be true.

However, we can project the gradient to the set of feasible descent directions
$$ \{d\in \mathbb R^n: Ad=0\},$$
which now is a linear subspace.

### Projection (投影)

Let $a\in \mathbb R^n$ be a vector and let $L$ be a linear subspace of $\mathbb R^n$. Now, the following are equivalent
* $a^P$ is the projection of $a$ on $L$, ($a^P$是$a$在$L$上的投影)
* $\{a^P\} = \operatorname{argmin}_{l\in L}\|a-l\|$, (与$L$上所有向量的距离中，$a$到$a^P$的距离最短)and
* $a^P\in A$ and $(a-a^P)^Tl=0$ for all $l\in L$ ($a-a^P$与所有$L$上的向量$l$正交).

### Projected gradient (投影梯度)

The projection of the gradient $\nabla f(x)$ on the set $\{d\in \mathbb R^n: Ad=0\}$ is denoted by $\nabla f(x)^P$ and called the __projected gradient__. 

Now, given some conditions, the projected gradient gives us a feasible descent direction.

### How to compute the projected gradient?

Basically, the optimization problem that we have to solve is
$$
\min\quad \|\nabla f(x)-d\|\\
\text{s.t. }\quad Ad=0.
$$

Since it is equivalent to minimize the square of the objective function $\sum_{i=1}^n \nabla_i f(x)^2+d_i^2-2\nabla_i f(x)d_i$, we can see that the problem is a quadratic problem with inequality constraints,
$$
\min \frac12 d^TId-\nabla f(x)^Td\\
\text{s.t. }Ad=0
$$
which means that we just need to solve the system of equations (see e.g., [Quadratic_programming](https://en.wikipedia.org/wiki/Quadratic_programming#Equality_constraints))
$$
\left[
\begin{array}{cc}
I&A^T\\
A&0
\end{array}
\right] 
\left[\begin{align}d\\\lambda\end{align}\right]
= \left[ 
\begin{array}{c}
\nabla f(x)\\
0
\end{array}
\right],
$$
where I is the identity matrix, and $\lambda$ are the KKT multipliers.

Here is a function for __projecting a vector to a linear space defined by $Ax=0$__:

In [ ]:
import numpy as np
def project_vector(A,vector):
    #convert A into a matrix
    A_matrix = np.matrix(A)
    #construct the "first row" of the matrix [[I,A^T],[A,0]]
    left_matrix_first_row = np.concatenate((np.identity(len(vector)),\
                                            A_matrix.transpose()), axis=1)
    #construct the "second row" of the matrix
    left_matrix_second_row = np.concatenate((A_matrix,np.matrix(np.zeros([len(A),\
                                            len(vector)+len(A)-len(A[0])]))), axis=1)
    #combine the whole matrix by combining the rows
    left_matrix = np.concatenate((left_matrix_first_row,left_matrix_second_row),axis = 0)
    #Solve the system of linear equalities from the previous page
    return np.linalg.solve(left_matrix, \
                           np.concatenate((np.matrix(vector).transpose(),\
                                           np.zeros([len(A),1])),axis=0))[:len(vector)]

In [ ]:
A = [[1,0,0],[0,1,0]]
gradient = [1,1,1]
project_vector(A,gradient)

## Example
Say we study an optimization problem
$$
\begin{align}
\min \qquad& x_1^2+x_2^2+x_3^2\\
\text{s.t.}\qquad &x_1+x_2=3\\
    &x_1+x_3=4.
\end{align}
$$
Let us project a gradient from a feasible point $x=(1,2,3)$.

In [ ]:
import ad
A = [[1,1,0],[1,0,1]]
gradient = ad.gh(lambda x:x[0]**2+x[1]**2+x[2]**2)[0]([1,2,3])
d = project_vector(A,[-i for i in gradient])
print d

__Here $d$ is a feasible direction since__:

In [ ]:
np.matrix(A)*d

__$d$ is also a descent direction since__:

In [ ]:
def f(x):
    return x[0]**2+x[1]**2+x[2]**2
alpha = 0.001
print "Value of f at [1,2,3] is "+str(f([1,2,3]))
x_mod= np.array([1,2,3])+alpha*np.array(d).transpose()[0]
print "Value of f at [1,2,3] + alpha*d is "+str(f(x_mod))
print "Gradient dot product direction (i.e., directional derivative) is " \
+ str(np.matrix(ad.gh(f)[0]([1,2,3])).dot(np.array(d)))

Finally, let's try projected gradient descent method to solve the optimization problem:

In [ ]:
import numpy as np
import ad
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        gradient = ad.gh(f)[0](x)
        grad_proj = project_vector(A,[-i for i in gradient])#The only changes to steepest..
        grad_proj = np.array(grad_proj.transpose())[0] #... descent are here!
#        import pdb; pdb.set_trace()
        x = x+grad_proj*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [ ]:
f = lambda x:x[0]**2+x[1]**2+x[2]**2
A = [[1,1,0],[1,0,1]]
start = [1,2,3]
(x,f_val,steps) = projected_gradient_method(f,A,start,0.6,0.000001)

At the end, check the result to see whether the objective value is smaller and if the solution is feasible.

In [ ]:
print x
print f(x)
print f([1,2,3])
print np.matrix(A)*(np.matrix(x).transpose())